## SpanBERT for coreference resolution

documentation: https://github.com/mandarjoshi90/coref

In [1]:
!pwd

/home/akvallapuram/FYP-19-20/notebook/modelling/coref_resolution


In [2]:
# as per the documentation
!export data_dir='../../data'

### Preprocessing

Need to process a document to be resolved into jsonline files as described in the documentation.

In [3]:
import sys
# get sample article -> Not Working!
sys.path.append('../../../')
from src.preparation.data_loading import read_dossier

In [4]:
articles = read_dossier.read_dossier()
dos = articles[0]

In [5]:
# BERT Tokenizer
sys.path.append('../../../models/coref/')
from bert import tokenization
tokenizer = tokenization.FullTokenizer(vocab_file='../../../models/bert_large/vocab.txt', do_lower_case=False)

In [6]:
sents = list(filter(lambda x: len(x) > 3, dos.split('.')))
sents = list(map(lambda x: tokenizer.tokenize(x.strip()) + ['.'], sents))
sents[0] = ["[CLS]"] + sents[0]
sents[-1].append("[SEP]")

In [7]:
def get_subtoken_map(sent_list):
    subtoken_map = list()
    i = -1
    for sent in sent_list:
        for t in range(len(sent)):
            if (not sent[t].startswith('#') \
            and sent[t-1] != "[CLS]") \
            and sent[t] != "[SEP]":
                i += 1
            
            subtoken_map.append(i)
    return subtoken_map


# [item for sublist in [[i]*len(sents[i]) for i in range(len(sents))] for item in sublist]
def get_sentence_map(sent_list):
    sent_map = list()
    for i in range(len(sent_list)):
        sent_map.extend([i]*len(sent_list[i]))
    
    return sent_map

In [8]:
# create jsonline file
data = dict()
data['clusters'] = []
data['doc_key'] = 'nw'
data['sentences'] = sents
data['speakers'] = list(map(lambda x: ""*len(x), sents)) # setting to No speaker for every subword at the moment
data['sentence_map'] = get_sentence_map(sents)
data['subtoken_map'] = get_subtoken_map(sents)

In [9]:
# check the subtoken_map: correct
print(list(zip(sents[0], data['subtoken_map'][:len(sents[0])])))

[('[CLS]', 0), ('Speaking', 0), ('to', 1), ('a', 2), ('trusted', 3), ('com', 4), ('##pa', 4), ('##tri', 4), ('##ot', 4), ('in', 5), ('June', 6), ('2016', 7), ('sources', 8), ('A', 9), ('and', 10), ('B', 11), (',', 12), ('a', 13), ('senior', 14), ('Russian', 15), ('Foreign', 16), ('Ministry', 17), ('figure', 18), ('and', 19), ('a', 20), ('former', 21), ('top', 22), ('level', 23), ('Russian', 24), ('intelligence', 25), ('officer', 26), ('still', 27), ('active', 28), ('inside', 29), ('the', 30), ('K', 31), ('##rem', 31), ('##lin', 31), ('respectively', 32), (',', 33), ('the', 34), ('Russian', 35), ('authorities', 36), ('had', 37), ('been', 38), ('cult', 39), ('##ivating', 39), ('and', 40), ('supporting', 41), ('US', 42), ('Republican', 43), ('presidential', 44), ('candidate', 45), (',', 46), ('Donald', 47), ('T', 48), ('##R', 48), ('##UM', 48), ('##P', 48), ('for', 49), ('at', 50), ('least', 51), ('5', 52), ('years', 53), ('.', 54)]


In [10]:
# write data to a jsonline format file
import json

In [11]:
with open('Dossier_for_spanBERT_coref.jsonlines', 'w') as f:
    f.write(json.dumps(data))